In [95]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader,Docx2txtLoader, TextLoader, UnstructuredExcelLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GPT4AllEmbeddings,HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.schema import Document

import requests
from bs4 import BeautifulSoup
import os

In [97]:
def select_loader(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.pdf':
        return PyPDFLoader(file_path)
    elif ext == '.csv':
        return CSVLoader(file_path)
    elif ext == '.docx' or ext == '.doc':
        return Docx2txtLoader(file_path)
    elif ext == ".txt":
        return TextLoader(file_path)
    elif ext == ".xlsx":
        return UnstructuredExcelLoader(file_path,mode="elements")
    else:
        return None

def load_all_files(directory_path):
    combined_text = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            print(file)
            file_path = os.path.join(root, file)
            loader = select_loader(file_path)
            if loader:
                documents = loader.load()
                combined_text += documents
    
    return combined_text

def crawl_web(urls):
    combined_text = []
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            # Loại bỏ các phần không cần thiết như script, style
            for script in soup(["script", "style"]):
                script.decompose()
            text = soup.get_text(separator='\n')
            combined_text.append(text)
        except Exception as e:
            print(f"Không thể crawl {url}: {e}")
    return combined_text


In [98]:
directory_path = "data"
urls_to_crawl = [
    "https://thptbuithixuan.hcm.edu.vn/homegd3",
    'https://thptbuithixuan.hcm.edu.vn/gioi-thieu/truong-thpt-bui-thi-xuan-noi-giac-mo-duoc-chap-canh/ctmb/14042/190970',
]
web_documents = crawl_web(urls_to_crawl)
documents = load_all_files(directory_path)
processed_web_documents = [Document(page_content=doc, metadata={}) for doc in web_documents]


combined_documents = processed_web_documents + documents


Bảng tính không có tiêu đề.xlsx
data.pdf
info.txt
Nữ thần Mặt Trời và Mặt Trăng.docx
tkb-khoi-11_289202418.pdf
tkb-khoi-11_289202418.xlsx


In [99]:
combined_documents

[Document(metadata={}, page_content='\n\n\n\n\n\n\r\n\tTRƯỜNG THPT BÙI THỊ XUÂN\r\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nKhối THPT\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n Ngành Giáo Dục Và Đào Tạo Thành Phố Hồ Chí Minh\n TRƯỜNG THPT BÙI THỊ XUÂN\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGiới thiệu\nCông khai thông tin\nBiểu mẫu\nKế hoạch công tác\nTuyển sinh\nTin tức - sự kiện\nCải cách hành chính\nChính trị - tư tưởng\nY tế học đường\nHoạt động chuyên môn\nGÓC HỌC TẬP TRỰC TUYẾN\nTHÔNG BÁO\nHƯỚNG DẪN\nTHỜI KHÓA BIỂU\nliên hệ \n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGiới thiệu\n \n&nbspCông khai thông tin\n \n&nbspBiểu mẫu\nThi - kiểm tra\n \n&nbspKiểm tra học kỳ 1\n \n\xa0\xa0\n&nbspMa trận\n \n\xa0\xa0\n\xa0Đề - Đáp án\n \n\xa0\xa0\n&nbspVăn\n \n\xa0\xa0\n&nbspToán\n \n\xa0\xa0\n&nbspLý\n \n\xa0\xa0\n&nbspHóa\n \n\xa0\xa0\n&nbspTiếng Anh\n \n\xa0\xa0\n&nbspSinh\n \n\xa0\xa0\n&n

In [100]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunks = text_splitter.split_documents(combined_documents)

# Embeding
# embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

db = FAISS.from_documents(chunks, embedding_model)

c:\Users\Dell\anaconda3\envs\chat\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [101]:
import os
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(temperature=0, model="llama3-8b-8192")
# model_file = 'vinallama-7b-chat_q5_0.gguf'
# llm = CTransformers(
#         model=model_file,
#         model_type="llama",
#         max_new_tokens=1024,
#         temperature=0.01
#     )

# prompt = "Bạn hãy giới thiệu về bản thân"
# output = llm(prompt, max_tokens=50, temperature=0.5)


# In kết quả
# print(output["choices"][0]["text"])

In [102]:
template = """<|im_start|>system\nSử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói tôi không biết, tìm thông tin liên quan nhất và nói tôi không chắc\n
    {context}<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant"""
prompt = PromptTemplate(template = template, input_variables=["context", "question"])
llm_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type= "stuff",
        # retriever = db.as_retriever(search_kwargs = {"k":3}, max_tokens_limit=1024),
        retriever = db.as_retriever(search_kwargs={"k": 5, "fuzzy": True}, max_tokens_limit=1024),

        return_source_documents = False,
        chain_type_kwargs= {'prompt': prompt}

    )

In [103]:
question = "cho tôi xem thời khóa biểu một vài lớp được không"
response = llm_chain.invoke({"query": question})
print(response['result'])

I can help you with that!

According to the information provided, here are the time tables for two classes:

**Class 11A10**

* GVCN: Lê Cúc Anh
* Time table for the first semester of the 2024-2025 school year

| Day | 2 | 3 | 4 | 5 | 6 | 7 |
| --- | --- | --- | --- | --- | --- | --- |
| Morning | ChCờ - C.ANH | Hóa - T.PHI | TABN - TABN 2 | Toán - V.SƠN | KTPL - C.MINH | Lí - M.TRÍ |
| Afternoon | HĐTN - C.ANH | Hóa - T.PHI | N.Ngữ - A.TRÂM | Toán - V.SƠN | YOGA - YOGA 2 | Toán - V.SƠN |
| Afternoon | HĐTN - C.ANH | KTPL - C.MINH | N.Ngữ - A.TRÂM | Sử - H.THẢO | TABN - TABN 2 | N.Ngữ - A.TRÂM |

**Class 11A1**

* GVCN: Phạm Thị Thanh Thảo
* Time table for the first semester of the 2024-2025 school year

| Day | 2 | 3 | 4 | 5 | 6 | 7 |
| --- | --- | --- | --- | --- | --- | --- |
| Morning | ChCờ - T.THẢO | Lí - B.VÂN | YOGA - YOGA 1 | GDTC - V.TUẤN | KTPL - T.LUYẾN | Toán - Y.DUNG |
| Afternoon | HĐTN - T.THẢO | Sinh - T.NGA | GDQP - H.QUÂN | GDTC - V.TUẤN | HĐTN - T.LUYẾN | Toán - Y.D